## Khởi tạo Spark

In [1]:
import findspark
findspark.init()

import pyspark
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import count

spark = (SparkSession
         .builder
         .appName("Linear Regression")
         .getOrCreate())

## Đọc và load tập dữ liệu Boston housing

In [2]:
bostonDF = (spark.read
            .option("HEADER", True)
            .option("inferSchema", True)
            .csv("../Downloads/BostonHousing.csv")
           )

bostonDF.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

## Tập dữ liệu Boston housing

`crim`: per capita crime rate by town.

`zn`: proportion of residential land zoned for lots over 25,000 sq.ft.

`indus`: proportion of non-retail business acres per town.

`chas`: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

`nox`: nitrogen oxides concentration (parts per 10 million).

`rm`: average number of rooms per dwelling.

`age`: proportion of owner-occupied units built prior to 1940.

`dis`: weighted mean of distances to five Boston employment centres.

`rad`: index of accessibility to radial highways.

`tax`: full-value property-tax rate per 10,000 dollars.

`ptratio`: pupil-teacher ratio by town.

`black`: $1000(Bk - 0.63)^2$ where $Bk$ is the proportion of blacks by town.

`lstat`: lower status of the population (percent).

`medv`: median value of owner-occupied homes in 1000 dollars.

Spark khác với nhiều machine learning framework khác ở chỗ ta cần huấn luyện mô hình của mình trên một cột duy nhất có chứa một vectơ gồm tất cả các feature (attribute) mà ta quan tâm. Ta sẽ chuẩn bị dữ liệu bằng cách tạo một cột có tên `features` gồm các thuộc tính `rm` (average number of rooms), `crim` (crime rate),  và `lstat` (lower status of the population).

Thêm cột `features` vừa tạo vào data frame sử dụng `VectorAssembler`:

In [3]:
from pyspark.ml.feature import VectorAssembler

featureCols = ["rm", "crim", "lstat"]
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")
bostonFeaturizedDF = assembler.transform(bostonDF)

bostonFeaturizedDF.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[6.575,0.00632,4.98]|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[6.421,0.02731,9.14]|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[7.185,0.02729,4.03]|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[6.998,0.03237,2.94]|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[7.147,0.06905,5.33]|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7| [6.43,0.02985,5.21]|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

Import thư viện linear regression và thiết lập output là thuộc tính `medv`, input là `features`. Tham khảo thêm về các tham số của thư viện LinearRegression ở [đây](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=vectorassembler#pyspark.ml.regression.LinearRegression).

In [4]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol="medv", featuresCol="features")

Fit mô hình với data

In [5]:
lrModel = lr.fit(bostonFeaturizedDF)

Xem các tham số của mô hình sau khi đã fit với data

In [6]:
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))

Coefficients: 5.2, -0.1, -0.6
Intercept: -2.6


**Ý nghĩa**: $predicted\_medv = (5.2 * rm) - (0.1 * crim) - (0.6 * lstat) - 2.6$

hay $predicted\_home\_value = (5.2 * number\_of\_rooms) - (0.1 * crime\_rate) - (0.6 * lower\_class) - 2.6$

Xem $R^2$ score

In [7]:
print("R^2 score: {}".format(lrModel.summary.r2))

R^2 score: 0.6458520515781128


**Ý nghĩa**: mô hình có thể giải thích khoảng $0.65 \%$ sự biến thiên (variance) trong dữ liệu. 

Xem một số chỉ số khác

In [8]:
lrModelSummary = lrModel.summary

print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))
print("Coefficient Standard Errors: " + str(lrModelSummary.coefficientStandardErrors))
print("T Values: " + str(lrModelSummary.tValues))
print("P Values: " + str(lrModelSummary.pValues))

RMSE score: 5.4678160740273904
Coefficient Standard Errors: [0.4420347151349426, 0.032022216026543225, 0.047669471406803186, 3.1660227928472815]
T Values: [11.80213848770088, -3.214670921988156, -12.135352093519293, -0.809296451597215]
P Values: [0.0, 0.0013900026454840564, 0.0, 0.41872805807610836]


Các chỉ số `Standard Errors`, `t-values`, `p-values` liên quan đến ý nghĩa thống kê (statistically significant) của các hệ số (tham số) của mô hình. Xem thêm về diễn giải của các chỉ số này ở [đây](https://dss.princeton.edu/online_help/analysis/interpreting_regression.htm) hoặc [đây](https://boostedml.com/2019/06/linear-regression-in-r-interpreting-summarylm.html).

Chọn ra 10 dòng dữ liệu đầu tiên (xem như nó là dữ liệu mới). Ta sẽ xem dự đoán của mô hình trên 10 dòng dữ liệu này và cho sánh nó với giá trị thực tế.

In [9]:
subsetDF = (bostonFeaturizedDF
            .limit(10)
            .select("features", "medv")
           )

subsetDF.show()

+--------------------+----+
|            features|medv|
+--------------------+----+
|[6.575,0.00632,4.98]|24.0|
|[6.421,0.02731,9.14]|21.6|
|[7.185,0.02729,4.03]|34.7|
|[6.998,0.03237,2.94]|33.4|
|[7.147,0.06905,5.33]|36.2|
| [6.43,0.02985,5.21]|28.7|
|[6.012,0.08829,12...|22.9|
|[6.172,0.14455,19...|27.1|
|[5.631,0.21124,29...|16.5|
|[6.004,0.17004,17.1]|18.9|
+--------------------+----+



Sử dụng phương thức `transform` trên mô hình được huấn luyện để xem dự đoán của nó.

`lrModel` là một estimator, ta có thể biến đổi dữ liệu bằng cách sử dụng phương thức `.transform()` của nó.

In [10]:
predictionDF = lrModel.transform(subsetDF)

predictionDF.show()

+--------------------+----+------------------+
|            features|medv|        prediction|
+--------------------+----+------------------+
|[6.575,0.00632,4.98]|24.0|28.857717647784423|
|[6.421,0.02731,9.14]|21.6|25.645644850540144|
|[7.185,0.02729,4.03]|34.7| 32.58746300992212|
|[6.998,0.03237,2.94]|33.4| 32.24191904275643|
|[7.147,0.06905,5.33]|36.2| 31.63288834584224|
| [6.43,0.02985,5.21]|28.7|  27.9657852461671|
|[6.012,0.08829,12...|22.9| 21.60241460459668|
|[6.172,0.14455,19...|27.1|18.543911230275796|
|[5.631,0.21124,29...|16.5| 9.478596352794202|
|[6.004,0.17004,17.1]|18.9| 18.85073477002384|
+--------------------+----+------------------+



Bây giờ, ta thử dự đoán giá nhà cho một điểm dữ liệu mới của một ngôi nhà 6 phòng ngủ (`rm = 6`), với tỷ lệ tội phạm là 3.6 (`crim = 3.6`), và tầng lớp có thu nhập thấp hơn trung bình là 12% (`lstat = 12`). Theo công thức ở trên, mô hình sẽ dự đoán giá nhà khoảng 21.

In [11]:
from pyspark.ml.linalg import Vectors

data = [(Vectors.dense([6., 3.6, 12.]), )]              # Tạo new data point
predictDF = spark.createDataFrame(data, ["features"])

lrModel.transform(predictDF).show()

+--------------+------------------+
|      features|        prediction|
+--------------+------------------+
|[6.0,3.6,12.0]|21.427061506649363|
+--------------+------------------+



## **Câu hỏi 1**: Huấn luyện mô hình và dùng nó để dự đoán cho dữ liệu mới

a. Tạo một mô hình linear regression mới với input là các biến `indus`, `age`, `dis`, và output là biến `medv`.

i. Tạo biến `newFeatures` cho huấn luyện mô hình

In [12]:
newFeatures = ["indus", "age", "dis"]
assembler = VectorAssembler(inputCols=newFeatures, outputCol="features")
bostonFeaturizedDF = assembler.transform(bostonDF)

bostonFeaturizedDF.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+-------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|           features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+-------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|   [2.31,65.2,4.09]|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6| [7.07,78.9,4.9671]|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7| [7.07,61.1,4.9671]|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4| [2.18,45.8,6.0622]|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2| [2.18,54.2,6.0622]|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7| [2.18,58.7,6.0622]|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|3

ii. Huấn luyện mô hình

In [13]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol="medv", featuresCol="features")

lrModel = lr.fit(bostonFeaturizedDF)

iii. In kết quả của mô hình (các hệ số và $R^2$ score )

In [14]:
print("R^2 score: {}".format(lrModel.summary.r2))

R^2 score: 0.28545147539354343


iv. Diễn giải các hệ số (`Coefficients`, `Intercept`), $R^2$ `score`, `RMSE score`, các `Standard Errors`, `t-values`, `p-values` của mô hình.

In [15]:
lrModelSummary = lrModel.summary

print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))
print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))
print("Coefficient Standard Errors: " + str(lrModelSummary.coefficientStandardErrors))
print("T Values: " + str(lrModelSummary.tValues))
print("P Values: " + str(lrModelSummary.pValues))

Coefficients: -0.7, -0.1, -1.5
Intercept: 43.0
RMSE score: 7.766715476913002
Coefficient Standard Errors: [0.07389071808161016, 0.019157338599964224, 0.27719285411308525, 2.315290077905648]
T Values: [-9.954031888835738, -4.907552574046968, -5.573530886805173, 18.58698533272279]
P Values: [0.0, 1.2485850908738882e-06, 4.079797522038575e-08, 0.0]


b. Dùng mô hình huấn luyện được để dự đoán cho các điểm dữ liệu mới có (`indus`, `age`, `dis`) lần lượt là `(11, 68, 4)`, `(6, 35, 2)`, `(19, 74, 8)`.

In [16]:
subset = [(11, 68, 4), 
            (6, 35, 2), 
            (19, 74, 8)]

subsetColumns = ["indus","age","dis"]
assembler = VectorAssembler(inputCols=subsetColumns, outputCol="features")
subsetDF = spark.createDataFrame(data=subset, schema=subsetColumns)
subsetDF= assembler.transform(subsetDF)


subsetDF.printSchema()
subsetDF  = subsetDF.select("features")
lrModel.transform(subsetDF).show()

root
 |-- indus: long (nullable = true)
 |-- age: long (nullable = true)
 |-- dis: long (nullable = true)
 |-- features: vector (nullable = true)

+---------------+------------------+
|       features|        prediction|
+---------------+------------------+
|[11.0,68.0,4.0]|22.370810825866748|
| [6.0,35.0,2.0]| 32.24076584405401|
|[19.0,74.0,8.0]|  9.74286069912749|
+---------------+------------------+



## **Câu hỏi 2**: Train/test split

Giữa $80\%$ dữ liệu để làm training set, $20\%$ còn lại để làm test set. Thế giá trị của `seed` là MSSV của bạn.

In [36]:
trainDF, testDF = bostonFeaturizedDF.randomSplit([.8, .2], seed=17133064)
print(f"Có {trainDF.cache().count()} dòng trong training set, và {testDF.cache().count()} dòng trong test set")

Có 401 dòng trong training set, và 105 dòng trong test set


a. Thực hiện lại các bước tương tự như hương dẫn ở trên 

i. Huấn luyện mô hình trên training set

In [37]:
assembler = VectorAssembler(inputCols=newFeatures, outputCol="features2")
assembler.transform(trainDF).show()

+-------+-----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+-------------------+-------------------+
|   crim|   zn|indus|chas|   nox|   rm| age|    dis|rad|tax|ptratio|     b|lstat|medv|           features|          features2|
+-------+-----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+-------------------+-------------------+
|0.01096| 55.0| 2.25|   0| 0.389|6.453|31.9| 7.3073|  1|300|   15.3|394.72| 8.23|22.0| [2.25,31.9,7.3073]| [2.25,31.9,7.3073]|
|0.01301| 35.0| 1.52|   0| 0.442|7.241|49.3| 7.0379|  1|284|   15.5|394.74| 5.49|32.7| [1.52,49.3,7.0379]| [1.52,49.3,7.0379]|
|0.01311| 90.0| 1.22|   0| 0.403|7.249|21.9| 8.6966|  5|226|   17.9|395.93| 4.81|35.4| [1.22,21.9,8.6966]| [1.22,21.9,8.6966]|
|0.01381| 80.0| 0.46|   0| 0.422|7.875|32.0| 5.6484|  4|255|   14.4|394.23| 2.97|50.0| [0.46,32.0,5.6484]| [0.46,32.0,5.6484]|
|0.01432|100.0| 1.32|   0| 0.411|6.816|40.5| 8.3248|  5|256|   15.1| 392.9| 3.95|31.6| [1.32,40.5,8.3248]| [1.3

In [38]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol="medv", featuresCol="features")
q2_lrModel = lr.fit(trainDF)

ii. In kết quả của mô hình (các hệ số (`Coefficients`, `Intercept`), `RMSE score`, và $R^2$ `score` ) trên training set

In [20]:
lrModelSummary = lrModel.summary


print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))


RMSE score: 7.766715476913002


b. Đánh giá mô hình trên test set (in ra `RMSE score`)

In [21]:
lrModelSet=  lr.fit(testDF)
lrModelSetSummary = lrModelSet.summary
print("RMSE score: {}".format(lrModelSetSummary.rootMeanSquaredError))

RMSE score: 6.306100517561223


## **Câu hỏi 3**: Nghiên cứu thêm về các tham số của `LinearRegression`

Điều chỉnh các tham số `regParam` (or `lambda`, regularization parameter) và `elasticNetParam` (or `alpha`, the ElasticNet mixing parameter, in range [0, 1]. For `alpha = 0`, the penalty is an `L2 penalty`. For `alpha = 1`, it is an `L1 penalty`.) của `LinearRegression`. Xem thêm ở [đây](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=vectorassembler#pyspark.ml.regression.LinearRegression).

- `regParam = 0`: ordinary least squares

- `regParam != 0, elasticNetParam = 0`: L2 (ridge regression)

- `regParam != 0, elasticNetParam = 1`: L1 (Lasso)

- `regParam != 0, elasticNetParam != 0`: L2 + L1 (elastic net)

a. `regParam = 0`: ordinary least squares

In [31]:
lr3a = LinearRegression(labelCol = "medv", featuresCol = "features", regParam =0)
print(lr3a)

LinearRegression_90dffa21a1a6


b. `regParam != 0, elasticNetParam = 0`: L2 (ridge regression)

In [32]:
lr3b=LinearRegression(labelCol = "medv", featuresCol = "features", regParam =0.2, elasticNetParam=0)
print(lr3b)

LinearRegression_1b3048bb3af6


c. `regParam != 0, elasticNetParam = 1`: L1 (Lasso)

In [33]:
lr3c = LinearRegression(labelCol="medv", featuresCol="features", regParam=0.3, elasticNetParam =1)
print(lr3c)

LinearRegression_7ba4f8a652e2


d. `regParam != 0, elasticNetParam != 0`: L2 + L1 (elastic net)

In [34]:
lr3d = LinearRegression(labelCol="medv", featuresCol="features", regParam=0.5, elasticNetParam =1)
print(lr3d)

LinearRegression_ad86ebfd0ae7


e. So sánh kết quả của các mô hình ở a, b, c, d.

In [35]:
lrModel3a = lr3a.fit(trainDF)
print("Coefficients 3a: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel3a.coefficients))
print("Intercept 3a: {0:.1f}".format(lrModel3a.intercept))
print("R^2 score 3a: {}".format(lrModel3a.summary.r2))
print("\n")
lrModel3b = lr3b.fit(trainDF)
print("Coefficients 3b: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel3b.coefficients))
print("Intercept 3b: {0:.1f}".format(lrModel3b.intercept))
print("R^2 score 3b: {}".format(lrModel3b.summary.r2))
print("\n")
lrModel3c = lr3c.fit(trainDF)
print("Coefficients 3c: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel3c.coefficients))
print("Intercept 3c: {0:.1f}".format(lrModel3c.intercept))
print("R^2 score 3c: {}".format(lrModel3c.summary.r2))
print("\n")
lrModel3d = lr3d.fit(trainDF)
print("Coefficients 3d: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel3d.coefficients))
print("Intercept 3d: {0:.1f}".format(lrModel3d.intercept))
print("R^2 score 3d: {}".format(lrModel3d.summary.r2))

Coefficients 3a: -0.8, -0.1, -1.8
Intercept 3a: 45.8
R^2 score 3a: 0.2972748067194765


Coefficients 3b: -0.8, -0.1, -1.6
Intercept 3b: 44.0
R^2 score 3b: 0.2964364870755055


Coefficients 3c: -0.7, -0.1, -1.0
Intercept 3c: 38.8
R^2 score 3c: 0.2852467382416898


Coefficients 3d: -0.6, -0.0, -0.5
Intercept 3d: 34.1
R^2 score 3d: 0.26386335128051874
